In [70]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import mpld3
import warnings
import numpy as np

In [47]:

df = pd.read_csv('povertyindex.csv')

# Ensure positive marker sizes
df['GDP_Growth_Size'] = df['GDP_Growth'].abs() + 0.1  # Add small value to avoid zero

fig = px.scatter(
    df,
    x='Population_Density',
    y='MPI',
    color='Region' if 'Region' in df.columns else 'Country',
    size='GDP_Growth_Size',
    hover_name='Country',
    animation_frame='Year',
    animation_group='Country',
    facet_col='Region' if 'Region' in df.columns else None,
    title='Population Density vs. Multidimensional Poverty Index (MPI)',
    labels={
        'Population_Density': 'Population Density (per km²)',
        'MPI': 'Multidimensional Poverty Index',
        'GDP_Growth_Size': 'GDP Growth (absolute, %)',
        'accesstoelec': 'Access to Electricity (%)'
    }
)

fig.update_traces(marker=dict(line=dict(width=0.5, color='DarkSlateGrey')))
fig.update_layout(legend_title_text='Region', height=700)
fig.write_html('scatter_density_vs_mpi.html')


In [62]:

df = pd.read_csv('povertyindex.csv')

# Optional: filter data for reasonable values
elec_min, elec_max = 0, 100
gdp_min, gdp_max = -10, 15

filtered = df[
    (df['accesstoelec'] >= elec_min) & (df['accesstoelec'] <= elec_max) &
    (df['GDP_Growth'] >= gdp_min) & (df['GDP_Growth'] <= gdp_max) &
    (df['Year'].between(2000, 2022))
]

# Use 95th percentile to stretch the color scale
mpi_max = filtered['MPI'].quantile(0.95)

fig = px.choropleth(
    filtered,
    locations='Country Code',
    color='MPI',
    hover_name='Country',
    animation_frame='Year',  # <- Adds slider for year
    color_continuous_scale='Inferno',
    range_color=[0, mpi_max],
    title='Multidimensional Poverty Index (MPI) by Country (2000–2022)',
    labels={'MPI': 'MPI'}
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds="locations")
fig.write_html('choropleth_mpi_slider.html')


In [73]:

# Load data
df = pd.read_csv('povertyindex.csv')

# Optional: filter sensible ranges
elec_min, elec_max = 0, 100
gdp_min, gdp_max = -10, 15

# Apply initial filters
filtered = df[
    (df['accesstoelec'] >= elec_min) & (df['accesstoelec'] <= elec_max) &
    (df['GDP_Growth'] >= gdp_min) & (df['GDP_Growth'] <= gdp_max) &
    (df['Year'].between(2000, 2022)) &
    (df['MPI'].between(0, 1)) &
    (df['Population_Density'] > 0)
].copy()

# Calculate thresholds for mismatch
pop25 = filtered['Population_Density'].quantile(0.25)
pop75 = filtered['Population_Density'].quantile(0.75)
mpi25 = filtered['MPI'].quantile(0.25)
mpi75 = filtered['MPI'].quantile(0.75)

# Keep only mismatch countries
mismatch = filtered[
    (
        (filtered['Population_Density'] <= pop25) & (filtered['MPI'] >= mpi75)
    ) |
    (
        (filtered['Population_Density'] >= pop75) & (filtered['MPI'] <= mpi25)
    )
].copy()

# Use full color range for contrast
mpi_max = mismatch['MPI'].quantile(0.95)

# Create choropleth map
fig = px.choropleth(
    mismatch,
    locations='Country Code',
    color='MPI',
    hover_name='Country',
    hover_data={
        'Population_Density': ':.2f',  
        'MPI': ':.3f'                
    },
    animation_frame='Year',
    color_continuous_scale='Inferno',
    range_color=[0, mpi_max],
    title='Countries with MPI and Population Density Mismatch (2000–2022)',
    labels={'MPI': 'MPI'}
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds="locations")
fig.write_html('choropleth_mpi_mismatch.html')
fig.show()


In [69]:
import pandas as pd
import plotly.express as px
import warnings
import numpy as np

# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Load data
df = pd.read_csv('povertyindex.csv')

# Filter valid entries
df = df[
    (df['Year'].between(2000, 2022)) &
    (df['MPI'] >= 0) & (df['MPI'] <= 1) &
    (df['Population_Density'] > 0)
].copy()

# Remove outliers in Population_Density using percentiles
lower_bound = df['Population_Density'].quantile(0.05)
upper_bound = df['Population_Density'].quantile(0.95)

df = df[
    (df['Population_Density'] >= lower_bound) &
    (df['Population_Density'] <= upper_bound)
]

# Create animated scatter plot
fig = px.scatter(
    df,
    x='Population_Density',
    y='MPI',
    animation_frame='Year',
    hover_name='Country',
    size='Population_Density',
    color='MPI',
    color_continuous_scale='Inferno',
    range_y=[0, 1],
    size_max=40,
    title='Population Density vs. MPI (2000–2022)',
    labels={
        'Population_Density': 'Population Density (per km²)',
        'MPI': 'Multidimensional Poverty Index (MPI)'
    }
)

# Improve layout
fig.update_layout(
    xaxis_title='Population Density (per km²)',
    yaxis_title='Multidimensional Poverty Index (MPI)',
    coloraxis_colorbar=dict(title='MPI'),
    transition_duration=500
)

# Save and show
fig.write_html('animated_mpi_vs_density.html')
fig.show()


In [50]:

df = pd.read_csv('povertyindex.csv')
year = 2015
low_density = df['Population_Density'] < 20
high_mpi = df['MPI'] > 0.4
mismatch = low_density & high_mpi

filtered = df[df['Year'] == year].copy()
filtered['Mismatch'] = ['Mismatch' if m else 'Normal' for m in mismatch[filtered.index]]

fig = px.choropleth(
    filtered,
    locations='Country Code',
    color='Mismatch',
    hover_name='Country',
    color_discrete_map={'Mismatch': 'orange', 'Normal': 'lightblue'},
    title=f'Poverty-Density Mismatch Map ({year})'
)
fig.update_geos(showcoastlines=True, showland=True, fitbounds="locations")
fig.write_html('map_poverty_density_mismatch.html')


In [51]:

df = pd.read_csv('povertyindex.csv')
year = 2015
filtered = df[df['Year'] == year]

fig = px.scatter_3d(
    filtered,
    x='Population_Density',
    y='MPI',
    z='GDP_Growth',
    color='Region' if 'Region' in df.columns else 'Country',
    hover_name='Country',
    title='3D Relationship: Population Density, MPI, and GDP Growth'
)
fig.write_html('3d_density_mpi_gdp.html')
